<img src='https://www.anadronestarting.com/wp-content/uploads/intel-main_opt.png' width=50%>

# Lecture004 - 실시간 추론하기


<div align='right'>성  민  석 (Minsuk Sung)</div>
<div align='right'>이  인  구 (Ike Lee)</div>

<img src='https://gitcdn.xyz/cdn/Tony607/blog_statics/9ddd48eb623175fb734e63865ebe604351c64c61/images/openvino/openvino.png' width=50%>

---

## OpenVINO를 통해 실시간 추론

In [1]:
import os
# assert 'computer_vision_sdk' in os.environ['PYTHONPATH']

from PIL import Image
import numpy as np

try:
    from openvino import inference_engine as ie
    from openvino.inference_engine import IENetwork, IEPlugin
except Exception as e:
    exception_type = type(e).__name__
    print("The following error happened while importing Python API module:\n[ {} ] {}".format(exception_type, e))
    sys.exit(1)

In [2]:
def pre_process_image(image, img_height=224):
    # Model input format
    n, c, h, w = [1, 3, img_height, img_height]
    processedImg = cv2.resize(image,(h,w),interpolation=cv2.INTER_AREA)

    # Normalize to keep data between 0 - 1
    processedImg = (np.array(processedImg) - 0) / 255.0

    # Change data layout from HWC to CHW
    processedImg = processedImg.transpose((2, 0, 1))
    processedImg = processedImg.reshape((n, c, h, w))

    return image, processedImg

In [3]:
# Plugin initialization for specified device and load extensions library if specified.
plugin_dir = None
model_xml = './bin/mobilenetv2.xml'
model_bin = './bin/mobilenetv2.bin'

In [4]:
# Devices: GPU (intel), CPU, MYRIAD
# plugin = IEPlugin("MYRIAD", plugin_dirs=plugi)
plugin = IEPlugin("CPU")

/home/intel/test/test2/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
# Read IRn_dir
net = IENetwork(model=model_xml, weights=model_bin)
assert len(net.inputs.keys()) == 1
assert len(net.outputs) == 1
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

/home/intel/test/test2/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  
/home/intel/test/test2/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# Load network to the plugin
exec_net = plugin.load(network=net)
del net

In [7]:
import pickle

# Custom Labels
# with open('./dropthebee.txt', 'r') as f:
#     labels = f.readlines()
    
# labels = list(map(lambda x: x.replace('\n',''),labels))

with open('./bin/class20.pickle', 'rb') as f:
    labels = pickle.load(f)

In [8]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = capture.read()

    image, processedImg = pre_process_image(frame)
    res = exec_net.infer(inputs={input_blob: processedImg})
    # Access the results and get the index of the highest confidence score
    output_node_name = list(res.keys())[0]
    res = res[output_node_name]

    # Predicted class index.
    idx = np.argsort(res[0])[-1]

    prob = res[0][idx]*100

    info = 'Predicted : None'
    color = (0, 0, 0)

    if prob > 80:
        info = 'Predicted : {} ({:.2f}%)'.format(labels[idx].upper(), prob)
        color = (0, 0, 255)

    cv2.putText(frame, info, (10, 30), 0, 1, color, 2)

    cv2.imshow("VideoFrame", frame)

    if cv2.waitKey(1) > 0:
        break

capture.release()
cv2.destroyAllWindows()

/home/intel/test/test2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/intel/test/test2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/intel/test/test2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/intel/test/test2/lib/python3.6/site-packages/tensorflow/python/framework/dty